# Projekt na lobolatoria z Cyfrowego przetwarzania sygnałów i obrazów

## Grupa: 
#### Mikołaj Baran 241128,
#### Jakub Fałdasz 235730 <br>
Termin zajęć: Czwartek, 7:30-9:00 TN


W celu poprawnego działania programu należy posiadać zainstalowaną bibliotekę bokeh: `pip install bokeh`

Jeżeli opisane później sterowanie wykresami nie wyświetla się lub nie jest aktywne należy użyć następująch komend przed uruchomieniem Jupyter'a: <br>
`pip install ipywidgets` - w celu sprawdzenia czy pakiet z widgetami jest zainstalowyny <br>
`jupyter nbextension enable --py widgetsnbextension` - aby włączyć obsługę widgetów <br> 

Jupyter Notebook sam powinien pobrać i uruchomić obsługę widgetów jednak jeżeli wystąpiły błędy przy instastacji możliwe jest że trzeba będzie je włączyć ręcznie.

In [1]:
import pandas as pd
import numpy as np
from scipy import signal
import re
import math
from datetime import timedelta
from ipywidgets import interact, fixed, Layout
import ipywidgets as widgets
from IPython.display import Javascript, display
from bokeh.plotting import show, figure
from bokeh.io import output_notebook,  push_notebook
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models import ColumnDataSource
from bokeh.models.tools import WheelZoomTool
from bokeh.models.widgets import RangeSlider, DateRangeSlider
from bokeh.layouts import layout, row, column
from bokeh.models.ranges import DataRange1d
output_notebook()

Loading BokehJS ...

# Zadanie 1
Polegające na wczytaniu dowolnego badanego sygnału z pliku tekstowego i jego podgląd na wykresie.

Zaczynamy od podania ścieżki do danego pliku, jego częstotliwości próbkowania przy pomocy funkcji `input` oraz otwarcia go w celu analizy poprzez użycie funkcji `open` w trybie do odczytu.

In [2]:
path = input("Podaj sciezke do pliku .txt i nacisnij \"Enter\" \n")
file = open(path, 'r')
frequency = int(input("Podaj częstotliwość próbkowania i nacisnij \"Enter\" \n"))


Podaj sciezke do pliku .txt i nacisnij "Enter" 
C:\Users\Mikolaj\Desktop\ekg1.txt
Podaj częstotliwość próbkowania i nacisnij "Enter" 
1000


Ponieważ każdy z plików ma inną formę reprezentacji danych (w różnych dokumentach w wierszach jest inna ilość wartości) tworzymy słownik którego zadaniem jest mapowani nazwy danego pliku na kolejny słownik który odzwierciedla poprawną reprezentację dla danych.

In [3]:
files_mapping = {
    'ekg1.txt': {
        "column1": [],
        "column2": [],
        "column3": [],
        "column4": [],
        "column5": [],
        "column6": [],
        "column7": [],
        "column8": [],
        "column9": [],
        "column10": [],
        "column11": [],
        "column12": [],
        "x": [],
    },
    'ekg100.txt': {
        "column1": [],
         "x": [],
    },
    'ekg_noise.txt': {
        "column1": [],
         "x": [],
    }
}

Następnie dzielimy podaną przez użytkownika ściekżkę do pliku używając znaku `\` jako separatora w celu wyłuskania nazwy pliku z sygnałem. Po uzyskaniu nazwy do pliku możemy użyć wcześniej stworzonej mapy aby przygotować odpowiedni słownik do którego wczytamy dane z otwartego dokumentu.

Ponieważ plik `ekg_noise` jako jedyny zawiera informacje o czasie pomiaru, nie musimy go sami obliczać. Stąd gdy wczytamy plik wystarczy podzielić każdy wiersz na znaku spacji i wpisać informacje o czasie do słownika pod kluczem `x` a wartość `column1`. Czas całego sygnału to poprostu ostatnia wpisana wartość z pliku którą zapisujemy też w zmiennej `time_in_milliseconds`.

Pozostałe pliki wymagają nieco więcej pracy przy wczytaniu danych. Na początku również dzielimy każdy wiersz na znaku spacji a potem w zależności od ilości kolumn wpisujemy kolejene wartości do słownika pod kluczem `column + (numer wystapienia wartosci w wierszu)`. Zmienna `line_count` liczy ogólną ilość pomiarów. Dzięki podanej wcześniej częstotliwości możemy obliczyć okres `T=1/F` i wyrazić go w milisekundach oraz ogólny czas trwania sygnału. Następnie wpisujemy wartości dla naszej osi x. Są to po prostu numery kolejnych pomiarów * obliczony okres.

Z odpowiednio wczytanych danych tworzymy `DataFrame` z modułu `pandas` a z nigo następnie źródło danych które potrafi odczytać nasz wykres.

In [4]:
file_name = path.split('\\')[-1]
data_dict = files_mapping[file_name]
if file_name == "ekg_noise.txt":
    for line in file:
        values = [x for x in line.split(' ') if x]
        data_dict["x"].append(float(values[0]))
        data_dict["column1"].append(float(values[1]))
        
    time_in_milliseconds = data_dict['x'][-1]

else:
    line_count = 0  
    for line in file:
        values = line.strip().split(' ')
        line_count += 1
        for num in range(len(values)):
            data_dict["column"+str(num+1)].append(float(values[num]))

    T = 1/frequency * 1000
    time_in_milliseconds = T * line_count

    for i in range(line_count):
         data_dict["x"].append(i*T)

df = pd.DataFrame(data=data_dict)
source = ColumnDataSource(df)

Teraz tworzymy nasz wykres. Nadajemy mu odpowiednie wymiary, podpisy do osi oraz narzędzia które chcemy dadać do jego obługi.
Opis narzędzi znajduje się później.

Ustawiamy również aby narzędzie `wheel_zoom` było domyślnie włączone.

In [5]:
p = figure( plot_width=950, 
            plot_height=600,
            tools="pan,wheel_zoom,box_zoom,reset",
            x_axis_label='Czas podany w formacie [m:s:ms]',
            y_axis_label='Napięcie [V]')
    
p.toolbar.active_scroll = p.select_one(WheelZoomTool)

Następnie dodajemy linie (dane)  do naszego wykresu. Bierzemy klucze w naszym utworzonym słowniku i dodajemy do wykresu wszystkie linie poza ostanią ponieważ jest to nasza oś x. Ustawiamy również nasze źródło danych stworzone wcześniej.

W celu lepszej czytelności wykresu konfigurujemy skalowanie osi x tak aby zawsze pokazywała czas w formacie `minuty:sekundy:milisekundy`.

In [6]:
tmp = list(df.columns)
for key in tmp[0:-1]:
    p.line(x='x', y=key, source=source)
p.xaxis.formatter = DatetimeTickFormatter(  years = "%M:%S:%3N",
                                            months = "%M:%S:%3N",
                                            days = "%M:%S:%3N",
                                            hours = "%M:%S:%3N",
                                            hourmin= "%M:%S:%3N", 
                                            minutes="%M:%S:%3N",
                                            minsec ="%M:%S:%3N",  
                                            seconds = "%M:%S:%3N", 
                                            milliseconds = "%M:%S:%3N")

W celu zwiększenia czytelności wykresu ustawiamy aby domyślnie pokazane było ostatnie 10 sekund sygnału. Aby to uzyskać napierw skalujemy oś x a następnie przy pomocy funkcji `loc` wybieramy tylko interesujący nas fragment danych. Użytkownik przeglądając wykres może w każdej chwili zmienić zakres wartości na os x.

In [7]:
range_end = time_in_milliseconds
range_start = time_in_milliseconds - 10000 if time_in_milliseconds>10000 else 1
p.x_range.start = range_start
p.x_range.end = range_end
source.data =  df.loc[(df['x'] >=  range_start) & (df['x'] <= range_end)]

Następnie tworzymy UI do naszych wykresów. Dwa pola tekstowe do wprowadzanie zakresu dla osi X oraz przycisk do ich zatwierdzenia. Do przycisku przypisujemy aby po kliknięciu wykonywała się metoda `on_change_button_clicked`. Najpierw czyta ona wpisany w okenka tekst a dalej dzieli go na minuty, sekundy i milisekundy jednocześnie zmieniając typ `string` na `int`. Potem zamienia to wszystko na wartość w milisekundach. Zakres ustawiany jest tak jak opisano powyżej przy użyciu metody `loc`. W razie niepowodzenia użytkownik zostanie poinformowany komunikatem.

Metoda `push_notebook` znajduje się tu po to aby przekazać kontrolę nad wykresem oprogramowaniu Jupyter.

In [8]:
layout = Layout(width='500px', height='40px')
start_input_box = widgets.Text(
        value = "",
        placeholder='Podaj czas w formacie m:s:ms (np. 00:05:123)',
        description='Początek przedzialu czasowego:',
        disabled=False,
        layout=layout,
        style = {'description_width': 'initial'}
    )
end_input_box = widgets.Text(
        value = "",
        placeholder='Podaj czas w formacie m:s:ms (np. 10:45:500)',
        description='Koniec przedzialu czasowego:',
        disabled=False,
        layout=layout,
        style = {'description_width': 'initial'}
    )
change_range_btn = widgets.Button(
        description="Zmień przedział czasu",
        layout=Layout(width='200px', height='40px'),
    )
output1 = widgets.Output()
def on_change_button_clicked(b):
    with output1:
        try:
            range_start_values = [ int(x) for x in start_input_box.value.split(':') ]
            range_end_values = [ int(x) for x in end_input_box.value.split(':') ]
            start = timedelta(minutes = range_start_values[0], seconds = range_start_values[1], milliseconds = range_start_values[2])
            stop = timedelta(minutes = range_end_values[0], seconds = range_end_values[1], milliseconds = range_end_values[2])
            source.data =  df.loc[(df['x'] >=  (start.total_seconds() * 1000)) & (df['x'] <= (stop.total_seconds() * 1000))]
            p.x_range.start = start.total_seconds() * 1000
            p.x_range.end = stop.total_seconds() * 1000
            push_notebook()
        except Exception as e:
            print(e)
            print("Nieprawidłowy format czasu")
change_range_btn.on_click(on_change_button_clicked)

Po odczytaniu pliku, stworzeniu modelu danych i wykresu możemy go pokazać przy użyciu funkcji `show` a UI pokazyjemy przy pomocy funkcji `display`. <br>
Narzędzia jakimi dysponuje użytkownik to kolejno: <br>
`pan` - umożliwia przesuwanie wykresu, <br>
`box_zoom` - pozwala zaznaczyć obszar wykresu który chcemy przybliżyć, <br>
`wheel_zoom` - umożliwia przybliżanie lub oddalanie wykresu kiedy używamy scrola wewnątrz wykresu lub skalowanie osi kiedy najedziemy myszką na daną oś i użyjemy scrolla tam, <br>
`reset` - przywraca widok wykresu do stanu pierwotnego. <br>

Znajdują się one po prawej stronie wykresu. Pionowa niebieska linia oznacza że dane narzędzie jest włączone.

Poniżej znajduje się sterowanie osią x. W pola tekstowe nalezy wpisać czas początkowy i końcowy w formacie minuty:sekundy:milisekundy a następnie zatwierdzić przyciskiem.

In [9]:
show(p, notebook_handle=True)
display(start_input_box,end_input_box, change_range_btn ,output1)

Text(value='', description='Początek przedzialu czasowego:', layout=Layout(height='40px', width='500px'), plac…

Text(value='', description='Koniec przedzialu czasowego:', layout=Layout(height='40px', width='500px'), placeh…

Button(description='Zmień przedział czasu', layout=Layout(height='40px', width='200px'), style=ButtonStyle())

Output()

Tutaj znajduje się funkcja która pozwala nam zapisać wartości które reprezentuje wykres. Możemy ustawić zakres na wykresie i poniższa funkcja zapisze tylko widocze dane do pliku `.txt`. Najpier należy podać ścieżke gdzie ma zostać utworzony lub nadpisany plik z danymi. Następnie nacisnąć przycisk zapisu. Tak samo jak w przypadku wyboru zakresów użytkownik zostanie poinformaowany czy operacja przebiegła pomyślnie. 

Cały zapis opiera się na użyciu metody `savetxt` z biblioteki `numpy`. Pierwszym parametrem jest utworzony wcześniej i obcięty `DataFrame` a drugi to format danych. W tym przypadku zapis `fmt=%d` oznacza że zapisujemy liczby w systemie dziesiętnym.

In [10]:
save_btn = widgets.Button(
    description="Zapisz wartości",
    layout=Layout(width='200px', height='40px'),
)

save_path = widgets.Text(
    value = "",
    placeholder='Podaj ścieżkę do zapisu wartości wraz z nazwą (np. C:\\Users\\Desktop\\zapis1.txt)',
    disabled=False,
    layout=Layout(width='600px', height='40px'),
    style = {'description_width': 'initial'}
)
output2 = widgets.Output()
display(save_path,save_btn, output2 )
def on_save_button_clicked(b):
    with output2:
        try:
            np.savetxt(save_path.value,df,  fmt='%d')
            print('Zapisano pod nazwą: ' + save_path.value.split('\\')[-1] + '\nw katalogu: ' + '\\'.join(save_path.value.split('\\')[0:-1]))
        except Exception as e:
            print("Zapis nie powiódł się")
    

save_btn.on_click(on_save_button_clicked)

Text(value='', layout=Layout(height='40px', width='600px'), placeholder='Podaj ścieżkę do zapisu wartości wraz…

Button(description='Zapisz wartości', layout=Layout(height='40px', width='200px'), style=ButtonStyle())

Output()

# Zadanie 2

Ponieważ w zadaniu drugim tworzymy aż 6 wykresów to w celu czytelności kodu deklarujemy funkcję dzięki której ustawimy skalowanie osi x aby reprezentowała czas w formacie minuty:sekundy:milisekundy. Tylko w przypadku dużego przybliżenia wykresu jednostka zmini się na mikrosekundy.

In [2]:
def set_x_axis_formatter(plot):
    plot.xaxis.formatter = DatetimeTickFormatter(  years = "%M:%S:%3N",
                                            months = "%M:%S:%3N",
                                            days = "%M:%S:%3N",
                                            hours = "%M:%S:%3N",
                                            hourmin= "%M:%S:%3N", 
                                            minutes="%M:%S:%3N",
                                            minsec ="%M:%S:%3N",  
                                            seconds = "%M:%S:%3N", 
                                            milliseconds = "%M:%S:%3N")

Dalej delkarujemy i tworzymy potrzebne zmienne. Pierwsza odpowiada za ilość próbek generowanych aby otrzymać sygnał. Następnie mamy częstotliwość próbkowania. Domyślnie ustawioną na `1500` jednak użytkownik może ją modyfikować dowolnie. Potem z częstotliwości próbkowania wyliczamy okres dla naszej sinusoidy. Następnie towrzymy wartości dla osi x przy pomocy funkcji `linespace` z biblioteki `numpy`. Jako argumenty podajemy początek i koniec przedziału a następnie ilosć generowanych próbek. Zmienna `t` to oś dla wykresów w dzienidzie czasu a `f` w dziedzinie częstotliwość które reprezentujemy w przedziale `[0, fs/2]` stąd generuejmy tu tylko połowe wartości. Na koniec deklarujemy dwie badane częstotliwosci podane w treści zadania.

In [12]:
Ns = 10000
Fs = 1500
T = 1/Fs
t = np.linspace(0, Ns*T, Ns)
f = np.linspace(0, 1/(2*T), Ns//2)
f1, f2 = 50, 60

Tutaj liczymy podane w zadaniu wartości. Na początku wyliczamy wartości sinus ze wzoru `2*pi*f*t` oraz ich kombinacje. Potem w użyciu funkcji `fft` z biblioteki `numpy` na podstawie tych sygnałow wyliczamy ich `Transformatę Fouriera` a na koniec dzięki funkcji `iff` wracamy do oryginalnego sygnału licząc `Odwrotną Transformatę Fouriera`

In [13]:
wave_50Hz = np.sin(2*np.pi*f1*t)
wave_60Hz = np.sin(2*np.pi*f2*t)
wave_50Hz_and_60Hz = wave_50Hz + wave_60Hz

fft_50Hz = np.fft.fft(wave_50Hz)
fft_50Hz_and_60Hz = np.fft.fft(wave_50Hz_and_60Hz)

ifft_50Hz = np.fft.ifft(fft_50Hz)
ifft_50Hz_and_60Hz = np.fft.ifft(fft_50Hz_and_60Hz)

Podobnie jak w zadaniu pierwszym tworzymy źródła danych dla naszych wykresów. Każdemu przypisujemy indywidualne wartości. Dla jeszcze nieprzetworzonych sygnałów oś x to czas. Dla `transformaty Fouriera` jest to częstotliwość, a dla `odwrotnej transformaty Fouriera` znowu czas. Oś y natomiast to dla nieprzetworzonych sygnałów wartości funkcji sinus które wyliczyliśmy wcześniej. Dla `transformaty Fouriera` jest to wartość bezwględna połowy wyliczonych przez funckcje `fft` wartości pomnożona przez `1/ilość próbek` w celu `normalizacji amplitudy`, a dla `odwrotnej transformaty Fouriera` wartości wyliczone przez funkcję `ifft` zostały zmienione na liczby rzeczywiste.

In [14]:
source_plot_50Hz = ColumnDataSource({'x': t,'wave': wave_50Hz })
source_plot50Hz_fft = ColumnDataSource({'x': f,'wave': (1/Ns * np.abs(fft_50Hz[0:Ns//2]))})
source_plot50Hz_ifft = ColumnDataSource({'x': t,'wave': ifft_50Hz.real })
    
source_plot_50_and_60Hz = ColumnDataSource({'x': t,'wave': wave_50Hz_and_60Hz })
source_plot_50_and_60Hz_fft = ColumnDataSource({'x': f,'wave': (1/Ns * np.abs(fft_50Hz_and_60Hz[0:Ns//2]))})
source_plot_50_and_60Hz_ifft = ColumnDataSource({'x': t,'wave': ifft_50Hz_and_60Hz.real })

Potem przechodzimy do tworzenia wykresów. Podobnie jak w zadaniu pierwszym nadajemy im wymiary, domyślne narzędzia, tytuł oraz podpisujemy osie. W celu zwiększenia czytelności zmniejszyliśmy ich wysokość.

In [15]:
plot_50Hz = figure(plot_width=950, 
               plot_height=215,
               tools="pan,wheel_zoom,box_zoom,reset",
               x_axis_label='Czas',
               y_axis_label='Amplituda',
               title = f'Sygnał o częstotliwości 50Hz')
plot50Hz_fft = figure(plot_width=950, 
               plot_height=215,
               tools="pan,wheel_zoom,box_zoom,reset",
               x_axis_label='Częstotliwość',
               y_axis_label='Amplituda',
               title = f'Transformata Fouriera dla sygnału o częstotliwości 50Hz')
plot50Hz_ifft = figure(plot_width=950, 
               plot_height=215,
               tools="pan,wheel_zoom,box_zoom,reset",
               x_axis_label='Czas',
               y_axis_label='Amplituda',
                title = f'Odwrotna transformata Fouriera dla sygnału o częstotliwości 50Hz')

plot_50_and_60Hz = figure(plot_width=950, 
               plot_height=215,
               tools="pan,wheel_zoom,box_zoom,reset",
               x_axis_label='Czas',
               y_axis_label='Amplituda',
               title = f'Kombinacja fal sinusoidalnych o częstotliwościach 50Hz i 60Hz')
    
plot_50_and_60Hz_fft = figure(plot_width=950, 
               plot_height=215,
               tools="pan,wheel_zoom,box_zoom,reset",
               x_axis_label='Czas',
               y_axis_label='Amplituda',
               title = f'Transformata Fouriera dla kombinacji fal sinusoidalnych o częstotliwościach 50Hz i 60Hz')             
plot_50_and_60Hz_ifft = figure(plot_width=950, 
               plot_height=215,
               tools="pan,wheel_zoom,box_zoom,reset",
               x_axis_label='Czas',
               y_axis_label='Amplituda',
               title = f'Odwrotna transformata Fouriera dla kombinacji fal sinusoidalnych o częstotliwościach 50Hz i 60Hz')

Używamy wcześniej zdeklarowanej funkcji aby ustawić formatowanie osi x.

In [16]:
set_x_axis_formatter(plot_50Hz)
set_x_axis_formatter(plot50Hz_ifft)
set_x_axis_formatter(plot_50_and_60Hz)
set_x_axis_formatter(plot_50_and_60Hz_ifft)

Dodajemy reprezentację wyliczonych danych w formie lini do wszystkich wykresów. Podajemy wartości `x`, `y` oraz wcześniej stworzone `źródła danych`. 

In [17]:
plot_50Hz.line(x='x',y='wave', source = source_plot_50Hz)
plot50Hz_fft.line(x='x',y='wave', source = source_plot50Hz_fft)
plot50Hz_ifft.line(x='x',y='wave', source = source_plot50Hz_ifft)

plot_50_and_60Hz.line(x='x',y='wave', source = source_plot_50_and_60Hz)
plot_50_and_60Hz_fft.line(x='x',y='wave', source = source_plot_50_and_60Hz_fft)
plot_50_and_60Hz_ifft.line(x='x',y='wave', source =source_plot_50_and_60Hz_ifft)

GlyphRenderer(id='1380', ...)

Tworzymy interfejs użytkownika do zmieniania częstotliwości próbkowania. Tworzymy pole tekstowe do wpisania wartości oraz przycisk do jej zatwierdzenia. <br>
Poniżej widzimy funkcję którą wywołujemy po kliknięciu na przycisk. Przelicza ona opisane wcześniej wartości transformat, oraz czasu w dokładnie ten sam sposób, a następnie zmienia związane z wykresami źródła danych.

In [18]:
fs_textbox = widgets.Text(
    value = str(Fs),
    placeholder='Podaj częstotliwość próbkowania',
    disabled=False,
    layout=Layout(width='600px', height='40px'),
    style = {'description_width': 'initial'}
)
change_fs_btn = widgets.Button(
    description="Zmień częstotliwość próbkowania",
    layout=Layout(width='600px', height='40px'),
    style = {'description_width': 'initial'},
)

output3 = widgets.Output()

def on_change_button_clicked(b):
    with output3:
        try:
            T = 1/int(fs_textbox.value)
            t = np.linspace(0, Ns*T, Ns)
            f = np.linspace(0, 1/(2*T), Ns//2)

            wave_50Hz = np.sin(2*np.pi*f1*t)
            wave_60Hz = np.sin(2*np.pi*f2*t)
            wave_50Hz_and_60Hz = wave_50Hz + wave_60Hz

            fft_50Hz = np.fft.fft(wave_50Hz)
            fft_50Hz_and_60Hz = np.fft.fft(wave_50Hz_and_60Hz)

            ifft_50Hz = np.fft.ifft(fft_50Hz)
            ifft_50Hz_and_60Hz = np.fft.ifft(fft_50Hz_and_60Hz)
                
            source_plot_50Hz.data = {'x': t, 'wave': wave_50Hz}
            source_plot50Hz_fft.data = {'x': f, 'wave': (2/Ns * np.abs(fft_50Hz[0:Ns//2]))}
            source_plot50Hz_ifft.data = {'x': t, 'wave': ifft_50Hz.real}
            source_plot_50_and_60Hz.data = {'x': t, 'wave': wave_50Hz_and_60Hz}
            source_plot_50_and_60Hz_fft.data = {'x': f, 'wave': (2/Ns * np.abs(fft_50Hz_and_60Hz[0:Ns//2]))}
            source_plot_50_and_60Hz_ifft.data = {'x': t, 'wave': ifft_50Hz_and_60Hz.real}           
            push_notebook()
        except Exception as e:
            print("Nieprawidłowa częstotliwość próbkowania")
            print(e)

change_fs_btn.on_click(on_change_button_clicked)

Z utworzonych wykresów budujemy jedną kolumnę do wyświetlenia aby wykresy były umieszczone czytelnie jeden pod drugim. Następnie pokazujemy je i oddajemy kontrole Jupyter'owi. Na koniec wyśiwetlamy UI dla użytkownika do zmiany `częstotliwości próbkowania`.

In [19]:
plots = column(plot_50Hz, plot50Hz_fft, plot50Hz_ifft, plot_50_and_60Hz, plot_50_and_60Hz_fft, plot_50_and_60Hz_ifft)
show(plots, notebook_handle=True)
display(fs_textbox,change_fs_btn,output3)

Text(value='1500', layout=Layout(height='40px', width='600px'), placeholder='Podaj częstotliwość próbkowania',…

Button(description='Zmień częstotliwość próbkowania', layout=Layout(height='40px', width='600px'), style=Butto…

Output()

# Zadanie 3

ekg100.txt to bardzo duży plik dlatego zadanie może działać bardzo powoli i może upłynąć kilka sekund zanim wykresy się pokażą.

Na samym początku prosimy użytkownika o podanie ścieżki do pliku `ekg100.txt`

In [ ]:
path = input("Podaj sciezke do pliku ekg100.txt i nacisnij \"Enter\" \n")
file = open(path, 'r')

Dalej podobnie jak w zadaniu pierwszym wczytujemy wartości z pliku jednocześnie licząc ile ich jest przy pomocy zmiennej `line_count`. Funkcja `strip` pomaga usunąć niepotrzebne spacje przed konwersją liczy w formacie `string` na `float`.

Potem na podstawie podanej częstotliwości `360 Hz` wyliczamy okres i zamieniamy go odrazu na wartość w milisekundach.

In [ ]:
wave = []
line_count = 0  
for line in file:
    value = line.strip()
    line_count += 1
    wave.append(float(value))

T = 1/360 * 1000

Dzięki okresowi oraz wiedzy ile mamy pomiarów uzupełniamy wartość osi x o odpowiednie wartości czasowe.

In [ ]:
t = []
for i in range(line_count):
    t.append(i*T)

Potem podobnie jak zadaniu drugim wyznaczamy wartości osi x w dziedznie częstotliwości dla wykresu reprezentującego transformatę Fouriera. Robimy to przy pomocy funkcji `linespace`.

Następnie wyliczamy `Transformatę Fouriera` oraz `Odrwrotną Transformatę Fouriera` przy pomocy funkcji `fft` oraz `ifft`.
Na koniec aby zobrazowac również że `IDFT` w istotcie różni się od orginalnego sygnału wyznaczamy te różnice.

In [ ]:
f = np.linspace(0, 1/(2*T), Ns//2)
fft = np.fft.fft(wave)
ifft = np.fft.ifft(fft)

diffrence = wave - ifft.real

Tworzymy źródła danych. Dla każdego wykresu osią x jest czas a dla transformaty Fouriera wyliczona częstotliwość. Osią y natomiast wyliczone wczesniej wartości a dla transformaty Fouriera tylko ich połowa pomnożona przez `1/ilość próbek` w celu normalizacji amplitudy.

In [ ]:
source_plot_main = ColumnDataSource({'x': t,'wave': wave})
source_fft = ColumnDataSource({'x': f,'wave': (1/Ns * np.abs(fft[0:Ns//2]))})
source_ifft = ColumnDataSource({'x': t,'wave': ifft.real})
source_diffrence =  ColumnDataSource({'x': t,'wave': diffrence})

Tworzymy wykresy dzięki funkcji `figure` przekazując jej odpowiednie kluczowe paramtery.

In [ ]:
plot_main = figure( plot_width=950, 
                    plot_height=215,
                    tools="pan,wheel_zoom,box_zoom,reset",
                    x_axis_label='Czas',
                    y_axis_label='Amplituda',
                    title = f'ekg100.txt')
plot5_fft = figure( plot_width=950, 
                    plot_height=215,
                    tools="pan,wheel_zoom,box_zoom,reset",
                    x_axis_label='Częstotliwość',
                    y_axis_label='Amplituda',
                    title = f'Transformata Fouriera dla ekg100.txt')
plot_ifft = figure( plot_width=950, 
                    plot_height=215,
                    tools="pan,wheel_zoom,box_zoom,reset",
                    x_axis_label='Czas',
                    y_axis_label='Amplituda',
                    title = f'Odwrotna transformata Fouriera ekg100.txt')

plot_diffrence = figure(plot_width=950, 
                        plot_height=215,
                        tools="pan,wheel_zoom,box_zoom,reset",
                        x_axis_label='Czas',
                        y_axis_label='Amplituda',
                        title = f'Różnica między sygnałem a jego odwrotną transformatą Fouriera')

Powiązujemy utworzone xródła danych z punktami widocznymi na wykresie.

In [ ]:
plot_main.line(x='x',y='wave', source = source_plot_main)
plot5_fft.line(x='x',y='wave', source = source_fft)
plot_ifft.line(x='x',y='wave', source = source_ifft)
plot_diffrence.line(x='x',y='wave', source = source_diffrence)

Pokazujemy wykresy.

In [ ]:
plots = column(plot_main, plot5_fft, plot_ifft, plot_diffrence)
show(plots, notebook_handle=True)

Jak widać różnice między pierwotym sygnałem a odwrotną transofrmatą Fouriera istnieją jednak są one bardzo małe (zaczynają się dopiero na 15 miejscu po przecinku). 

# Zadanie 4

Celem ćwiczenia jest wyeliminowanie zakłóceń w sygnale zapisanym w pliku `ekg_noise.txt`. W tym celu zostanie użyty filtr `Butterwortha`.

Na początku należy otworzyć plik zawierający próbki zakłóconego sygnału `ekg_noise.txt`.

In [3]:
file = open('ekg_noise.txt', 'r')

Podobnie jak w poprzednich zadaniach odczytane próbki sygnału zostaną przypisane do odpowiednich zmiennych, tak aby można było później utworzyć odpowiedni wykres.

W zmiennej `time_in_milliseconds` został przypisany całkowity czas trwania sygnału. Zmienna `samples_count` przechowuje informacje o ilości próbek sygnału w pliku `ekg_noise.txt`.

In [4]:
fs = 360
file = open('ekg_noise.txt', 'r')

wave = {
    'x': [],
    'column1': []
}

for line in file:
    values = [x for x in line.split(' ') if x]
    wave["x"].append(float(values[0]))
    wave["column1"].append(float(values[1]))
        
time_in_milliseconds = wave['x'][-1]
samples_count = math.ceil(fs * time_in_milliseconds)
df = pd.DataFrame(data=wave)
source = ColumnDataSource(df)


Następnie tworzymy okno wykresu, przeskalowujemy oś x i wypełniamy `plot` odpowiednimi danymi.  

In [5]:
plot = figure( plot_width=950, 
            plot_height=250,
            tools="pan,wheel_zoom,box_zoom,reset",
            x_axis_label='Czas',
            y_axis_label='Amplituda [V]',
            title = f'Sygnał ekg_noise.txt')

plot.toolbar.active_scroll = plot.select_one(WheelZoomTool)

set_x_axis_formatter(plot)    

plot.line(x='x', y='column1', source = source)

GlyphRenderer(id='1044', ...)

Końcowym efektem jest wyświetlony wykres.

In [6]:
show(plot, notebook_handle=True)


Ostatnią rzeczą jaką należy wykonać w pierwszej części zadania jest określenie charakterystyki częstotliwościowej zakłóconego sygnału, co osiągniemy poprzez wykonanie transformacji fouriera na wczytanym sygnale. 

Zaczynamy od wyliczenia potrzebnych zmiennych takich jak okres `T`, ilość próbek `sample_count`. Zmienna `f` przechowuje wartości na osi x wykresu widma. Wyliczona została podobnie jak w poprzednich zadanach za pomocą funkcji `np.linspace()`.

In [7]:
T = 1/fs * 1000 # okres w milisekundach
sample_count = len(wave['x'])
f = np.linspace(0, 1/(2*T), sample_count//2)

Transformacja została przeprowadzona z użyciem funkcji `np.fft.fft()`. 

Po wykonaniu FFT widmo sygnału zostanie przedstawione na wykresie.

In [8]:
fft = np.fft.fft(wave['column1'])

source_fft = ColumnDataSource({'x': f,'wave': (1/sample_count * np.abs(fft[0:sample_count//2]))})

plot_fft = figure( plot_width=950, 
            plot_height=250,
            tools="pan,wheel_zoom,box_zoom,reset",
            x_axis_label='Czestotliwosc',
            y_axis_label='Amplituda',
            title = f'Widmo sygnału ekg_noise.txt')

plot_fft.line(x='x',y='wave', source = source_fft)
show(plot_fft, notebook_handle=True)

W kolejnej części zadania należy zaprojektować dolnoprzepustowy filtr Butterwortha i użyć go do filtracji sygnału `ekg_noise.txt`. Częstotliwość graniczna filtru wynosi `fc = 60`. Najpierw normalizujemy `fc`, co zostało przedstawione w drugiej linijce poniższego listingu `w = fc / (fs/2)`. Następnie utworzony został dolnoprzepustowy filtr Butterwortha 5 rzędu przy pomocy funkcji `scipy.signal.butter(N, Wn, btype='low')`.

Dodatkowo na następnym wykresie została przedstawiona charakterystyka filtru (zależność tłumienia od częstostliwości).

Następnie utworzony został nowy sygnał poprzez "przepuszczenie" sygnału `ekg_noise.txt` przez uprzednio utworzony filtr. Wyznaczone zostało również widmo sygnału, różnica między oryginalnym sygnałem oraz widmo różnicy.

In [9]:
fc = 60
w = fc / (fs/2)
b, a = signal.butter(5, w, 'low')
w, h = signal.freqz(b, a)

freq_response = ColumnDataSource({'x': (fs * 0.5 / np.pi) * w,'wave': abs(h)})

plot_freq_response = figure( plot_width=950, 
            plot_height=250,
            tools="pan,wheel_zoom,box_zoom,reset",
            x_axis_label='Częstotliwość [Hz]',
            y_axis_label='Amplituda',
            title = f'Charakterystyka filtru')

plot_freq_response.line(x='x',y='wave', source = freq_response)
show(plot_freq_response, notebook_handle=True)

filtered_signal = signal.filtfilt(b, a, wave['column1'])
fft_filtered = np.fft.fft(filtered_signal)
difference = wave['column1'] - filtered_signal
fft_difference = np.fft.fft(difference)


Odfiltrowany sygnał, jego widmo, różnica między oryginalnym sygnałem, a przefiltrowanym oraz widmo różnicy zostaną wyświetlone na kolejnych wykresach.


In [10]:
source_filtered = ColumnDataSource({'x': wave['x'],'wave': filtered_signal})
source_filtered_fft = ColumnDataSource({'x': f,'wave': (1/sample_count * np.abs(fft_filtered[0:sample_count//2]))})
source_difference = ColumnDataSource({'x': wave['x'] ,'wave': difference })
source_difference_fft = ColumnDataSource({'x': f,'wave': (1/sample_count * np.abs(fft_difference[0:sample_count//2]))})


plot_filtered = figure( plot_width=950, 
            plot_height=250,
            tools="pan,wheel_zoom,box_zoom,reset",
            x_axis_label='Czas',
            y_axis_label='Amplituda',
            title = f'Odfiltrowany sygnał ekg_noise.txt')

plot_filtered_fft = figure( plot_width=950, 
            plot_height=250,
            tools="pan,wheel_zoom,box_zoom,reset",
            x_axis_label='Czestotliwosc',
            y_axis_label='Amplituda',
            title = f'Widmo odfiltrowanego sygnału ekg_noise.txt')

plot_difference = figure( plot_width=950, 
            plot_height=250,
            tools="pan,wheel_zoom,box_zoom,reset",
            x_axis_label='Czas',
            y_axis_label='Amplituda',
            title = f'Różnica między oryginalnym a przefiltrowanym sygnałem ekg_noise.txt (LOWPASS)')

plot_difference_fft = figure( plot_width=950, 
            plot_height=250,
            tools="pan,wheel_zoom,box_zoom,reset",
            x_axis_label='Czestotliwosc',
            y_axis_label='Amplituda',
            title = f'Widmo różnicy między oryginalnym a przefiltrowanym sygnałem ekg_noise.txt (LOWPASS)')

plot_filtered.line(x='x',y='wave', source = source_filtered)
plot_filtered_fft.line(x='x',y='wave', source = source_filtered_fft)
plot_difference.line(x='x',y='wave', source = source_difference)
plot_difference_fft.line(x='x',y='wave', source = source_difference_fft)


plots = column(plot_filtered, plot_filtered_fft, plot_difference, plot_difference_fft)
show(plots, notebook_handle=True)

W kolejnej części zadania na uprzednio przefiltrowany sygnał zostanie nałożony górnoprzepustowy filtr Butterwortha o częstotliwości granicznej `5 Hz` w celu eliminacji pływania linii izoelektrycznej. Całość zostanie zrealizowana w podobny sposób jak w przypadku filtru dolnoprzepustowego.

In [11]:
fc = 5
w = fc / (fs/2)
b, a = signal.butter(5, w, 'high')

highpass_filtered_signal = signal.filtfilt(b, a, filtered_signal)
fft_filtered_final = np.fft.fft(highpass_filtered_signal)
difference_final = wave['column1'] - highpass_filtered_signal
fft_difference_final = np.fft.fft(difference_final)

Otrzymany sygnał oraz jego widmo zostaną następnie wyświetlone na poniższych wykresach.

In [12]:
source_filtered_highpass = ColumnDataSource({'x': wave['x'],'wave': highpass_filtered_signal})
source_filtered_fft_final = ColumnDataSource({'x': f,'wave': (1/sample_count * np.abs(fft_filtered_final[0:sample_count//2]))})
source_difference_final = ColumnDataSource({'x': wave['x'] ,'wave': difference_final })
source_difference_final_fft = ColumnDataSource({'x': f,'wave': (1/sample_count * np.abs(fft_difference_final[0:sample_count//2]))})


plot_filtered_highpass = figure( plot_width=950, 
            plot_height=250,
            tools="pan,wheel_zoom,box_zoom,reset",
            x_axis_label='Czas',
            y_axis_label='Amplituda',
            title = f'Odfiltrowany sygnał ekg_noise.txt')

plot_filtered_fft_final = figure( plot_width=950, 
            plot_height=250,
            tools="pan,wheel_zoom,box_zoom,reset",
            x_axis_label='Czestotliwosc',
            y_axis_label='Amplituda',
            title = f'Widmo sygnału ekg_noise.txt')

plot_difference_final = figure( plot_width=950, 
            plot_height=250,
            tools="pan,wheel_zoom,box_zoom,reset",
            x_axis_label='Czas',
            y_axis_label='Amplituda',
            title = f'Różnica między oryginalnym a przefiltrowanym sygnałem ekg_noise.txt (HIGHPASS)')

plot_difference_final_fft = figure( plot_width=950, 
            plot_height=250,
            tools="pan,wheel_zoom,box_zoom,reset",
            x_axis_label='Czestotliwosc',
            y_axis_label='Amplituda',
            title = f'Widmo różnicy między oryginalnym a przefiltrowanym sygnałem ekg_noise.txt (HIGHPASS)')


plot_filtered_highpass.line(x='x',y='wave', source = source_filtered_highpass)
plot_filtered_fft_final.line(x='x',y='wave', source = source_filtered_fft_final)
plot_difference_final.line(x='x',y='wave', source = source_difference_final)
plot_difference_final_fft.line(x='x',y='wave', source = source_difference_final_fft)
plots = column(plot_filtered_highpass, plot_filtered_fft_final, plot_difference_final, plot_difference_final_fft)
show(plots, notebook_handle=True)